In [1]:
import os
import sys

sys.path.insert(0, '../')

In [2]:
import pandas as pd
import numpy as np

from src.models.model_common import select_features

/Users/sreejithkrishnanr/Tools/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train_data = pd.read_hdf('../data/processed/train_3600000000000.hd5', 'data', parse_dates=[1])

In [5]:
site_data = train_data.loc[train_data['SiteId'] == 2, :]
site_data.head()

,ForecastId,Timestamp,obs_id,SiteId,Value,Frequency,DayOfMonth,DayOfWeek,DayOfYear,DaysInMonth,...,ConsumptionPerSurfaceArea,ConsumptionPerTemperatureDiff,ConsumptionDailyMeanPerSurfaceArea,ConsumptionDailyMeanPerTemperatureDiff,ConsumptionWeeklyMeanPerSurfaceArea,ConsumptionWeeklyMeanPerTemperatureDiff,ConsumptionBiWeeklyMeanPerSurfaceArea,ConsumptionBiWeeklyMeanPerTemperatureDiff,ConsumptionMonthlyMeanPerSurfaceArea,ConsumptionMonthlyMeanPerTemperatureDiff
0,5,2013-01-01 01:00:00,4382312,2,30159.963708,3.600000e+12,1,1,1,31,...,4.945652,93.086308,7.071668,133.101861,13.777236,259.313025,14.116082,265.69073,14.529086,273.464223
1,5,2013-01-01 02:00:00,2016541,2,29793.536720,3.600000e+12,1,1,1,31,...,4.885565,91.955360,7.071668,133.101861,13.777236,259.313025,14.116082,265.69073,14.529086,273.464223
2,5,2013-01-01 03:00:00,78869,2,31682.412161,3.600000e+12,1,1,1,31,...,5.195304,97.785223,7.071668,133.101861,13.777236,259.313025,14.116082,265.69073,14.529086,273.464223
3,5,2013-01-01 04:00:00,1361014,2,29881.829185,3.600000e+12,1,1,1,31,...,4.900043,92.227868,7.071668,133.101861,13.777236,259.313025,14.116082,265.69073,14.529086,273.464223
4,5,2013-01-01 05:00:00,129169,2,30560.328940,3.600000e+12,1,1,1,31,...,5.011304,94.322003,7.071668,133.101861,13.777236,259.313025,14.116082,265.69073,14.529086,273.464223


In [ ]:
def create_lagged(x, y, groups, num_lags=24):
    fids = groups.groupby('ForecastId')['ForecastId'].first().values
    
    for fid in fids:
        x_f = x.loc[groups == fid, :]
        y_f = y.loc[groups == fid]
        

In [6]:
x, y, groups = select_features(site_data, 'h')

In [10]:
groups.value_counts()

31    964
30    964
11    964
10    964
9     964
40    964
8     964
39    964
7     964
38    964
6     964
37    964
5     964
36    964
35    964
34    964
33    964
12    964
13    964
14    964
23    964
29    964
28    964
27    964
26    964
25    964
24    964
22    964
15    964
21    964
20    964
19    964
18    964
17    964
16    964
32    964
Name: ForecastId, dtype: int64

In [12]:
from xgboost import XGBRegressor

regressor = XGBRegressor()

In [13]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(regressor, {
    'learning_rate': [0.10],
    'max_depth': [4],
    'n_estimators': [75],
    'colsample_bytree': [0.8],
    'colsample_bylevel': [0.7],
    'gamma': [0],
    'reg_alpha': [1.2],
    'reg_lambda': [1.2]
}, n_jobs=8, cv=8, scoring='neg_mean_squared_error')

gs.fit(x, y)

GridSearchCV(cv=8, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'learning_rate': [0.1], 'max_depth': [4], 'n_estimators': [75], 'colsample_bytree': [0.8], 'colsample_bylevel': [0.7], 'gamma': [0], 'reg_alpha': [1.2], 'reg_lambda': [1.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [11]:
gs.best_params_

{'colsample_bylevel': 0.7,
 'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 75,
 'reg_alpha': 1.2,
 'reg_lambda': 1.2}

In [15]:
gs.best_score_

-518270729.12947804